# Nutritional Composition Analysis

## 1. Setup and Imports

In [1]:
# Standard library
import os
from typing import Any
import sys

# Third-party libraries
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import seaborn as sns

# Local libraries
from util import print_col

# Plot styles
plt.rcdefaults()
plt.style.use("seaborn-v0_8-colorblind")
sns.set_palette("colorblind")

# Version info
print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")
print(f"PyArrow version:", pa.__version__)
print(f"Pandas version: {pd.__version__}")
print(f"Dask version: {dask.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Seaborn version: {sns.__version__}")


Python version: 3.12.11 (main, Jun  3 2025, 15:41:47) [Clang 17.0.0 (clang-1700.0.13.3)]
NumPy version: 2.3.2
PyArrow version: 21.0.0
Pandas version: 2.3.2
Dask version: 2025.7.0
Matplotlib version: 3.10.5
Seaborn version: 0.13.2


## 2. Data Loading


Load the dataset using Dask.


In [2]:
def load_dataset(file_path: str) -> dd.DataFrame:
    print(f"Loading the dataset...")

    # Load dataset using dask
    df = dd.read_parquet(file_path, engine="pyarrow", dtype_backend="pyarrow")

    return df

df = load_dataset("../eda/food.parquet")


Loading the dataset...


Based on our EDA findings, the ingredients column is very large; therefore, we will drop it in this full data analysis.


In [3]:
df = df.drop(columns="ingredients")


Print the dataset shape.


In [4]:
df_shape = df.shape[0].compute(), df.shape[1]
print(f"Dataset shape: {df_shape}")
print(f"Total cells: {df_shape[0] * df_shape[1]:,}")


Dataset shape: (3919113, 109)
Total cells: 427,183,317


Load the `data/column_analysis` dataset.


In [5]:
column_analysis_df = pd.read_csv("data/column_analysis.csv")
print(f"Dataset shape: {column_analysis_df.shape}")


Dataset shape: (109, 6)


Print the columns from `nutritional` category.


In [6]:
print(f"Category: \033[94mnutritional\033[0m\n")
for i, row in enumerate(column_analysis_df[column_analysis_df["category"] == "nutritional"].itertuples(index=False), start=1):
    print_col(i, row.column_name, sample=row.sample)


Category: nutritional

  1. minerals_tags                            | []
  2. no_nutrition_data                        | False
  3. nucleotides_tags                         | []
  4. nutrient_levels_tags                     | ['en:fat-in-high-quantity', 'en:saturated-fat-in-high-quantity', 'en:sugars-in-h...
  5. nutriments                               | [{'name': 'saturated-fat', 'value': 10.0, '100g': 10.0, 'serving': None, 'unit':...
  6. nutriscore_grade                         | e
  7. nutriscore_score                         | 25
  8. nutrition_data_per                       | 100g
  9. product_quantity_unit                    | g
 10. product_quantity                         | 350
 11. quantity                                 | 350 g
 12. serving_quantity                         | 1.0
 13. serving_size                             | 1g
 14. unknown_nutrients_tags                   | []
 15. vitamins_tags                            | []
 16. with_non_nutritive_sweeteners        

## 3. Nutrient Extraction


Extract nutrients and save into `data/nutrient_extract`.


In [7]:
def normalise_unit(unit: str) -> str:
    """
    Normalize unit labels without altering the values:
      - Mass-based nutrients (g, mg, µg, mcg) → 'g'
      - Energy (kcal, Cal, kJ) → 'kJ'
      - Others (IU, %, %DV, %vol, None) → keep as-is
    """
    if unit in ["g", "mg", "µg", "mcg"]:
        return "g"
    elif unit in ["kcal", "Cal", "kJ"]:
        return "kJ"
    else:
        return unit

def extract_nutrients_partitions(df: pd.DataFrame) -> pd.DataFrame:
    records = []
    for code, nutriment in zip(df["code"], df["nutriments"]):
        if nutriment is pd.NA:
            continue
        for nutrient in nutriment:
            if not (name := nutrient.get("name")):
                continue
            if not (value_100g := nutrient.get("100g")):
                continue
            if not (unit := nutrient.get("unit")):
                if name != "nova-group":
                    continue
                unit = ""
            records.append({
                "code": code,
                "name": name,
                "100g": value_100g,
                "unit": normalise_unit(unit),
            })
    return pd.DataFrame(records)

parts = df[["code", "nutriments"]].map_partitions(
    extract_nutrients_partitions,
    meta=pd.DataFrame({
        "code": pd.Series(dtype="str"),
        "name": pd.Series(dtype="str"),
        "100g": pd.Series(dtype="float64"),
        "unit": pd.Series(dtype="str"),
    }),
)

with ProgressBar():
    parts.to_parquet("data/nutrient_extract", engine="pyarrow", write_index=False, overwrite=True)


[########################################] | 100% Completed | 233.15 s


Load the `data/nutrient_extract` dataset.


In [8]:
nutrient_extract_df = dd.read_parquet("data/nutrient_extract", engine="pyarrow", dtype_backend="pyarrow")


Analyse nutrients extracted.


In [9]:
n_products = nutrient_extract_df["code"].nunique().compute()

print(f"Successfully processed {n_products:,} products.")
print(f"Found {nutrient_extract_df["name"].nunique().compute():,} unique nutrients.")


Successfully processed 2,991,066 products.
Found 2,209 unique nutrients.


Retain only the nutrients with at least 100 observations.


In [10]:
focus_nutrients = (
    nutrient_extract_df
    .assign(unit=nutrient_extract_df['unit'].fillna(''))
    .groupby(["name", "unit"])
    .size()
    .compute()
    .loc[lambda x: x >= 100]
    .sort_values(ascending=False)
)
print(f"Found {len(focus_nutrients)} nutrients with at least 100 observations.\n")


Found 120 nutrients with at least 100 observations.



In [11]:
nutrients_df = focus_nutrients.reset_index()
nutrients_df.columns = ["name", "unit", "count"]

nutrients_df["percent"] = nutrients_df["count"] / n_products * 100

print("\nTop 15 most complete nutrients:\n")
for i, row in enumerate(nutrients_df.sort_values("percent", ascending=False).head(15).itertuples(index=False), start=1):
    print_col(i, row.name, desc=f"{row.count:10,d} ({row.percent:5.2f}%)", sample=row.unit)



Top 15 most complete nutrients:

  1. energy                                   |  2,820,319 (94.29%) | kJ
  2. energy-kcal                              |  2,784,551 (93.10%) | kJ
  3. carbohydrates                            |  2,490,622 (83.27%) | g
  4. proteins                                 |  2,382,737 (79.66%) | g
  5. fat                                      |  2,301,562 (76.95%) | g
  6. sugars                                   |  2,183,241 (72.99%) | g
  7. sodium                                   |  2,162,812 (72.31%) | g
  8. salt                                     |  2,025,188 (67.71%) | g
  9. saturated-fat                            |  2,010,883 (67.23%) | g
 10. nova-group                               |  1,014,600 (33.92%) | 
 11. fiber                                    |    995,152 (33.27%) | g
 12. energy-kj                                |    368,607 (12.32%) | kJ
 13. calcium                                  |    287,832 ( 9.62%) | g
 14. iron                   

## 4. Nutrient Categorisation


Classify nutrients into different categories.


In [12]:
energy_terms = [
    "energy", "calorie", "kcal", "kj", "energy-from"
]

protein_terms = [
    "protein", "collagen"
]

fat_terms = [
    "fat", "cholesterol",
    # general groups
    "saturated-fat", "unsaturated-fat",
    "monounsaturated-fat", "polyunsaturated-fat",
    "trans-fat", "omega-3-fat", "omega-6-fat", "omega-9-fat",
    # specific fatty acids
    "alpha-linolenic-acid", "linoleic-acid", "oleic-acid",
    "docosahexaenoic-acid", "eicosapentaenoic-acid",
    "arachidonic-acid", "arachidic-acid", "behenic-acid"
]

sugars_terms = [
    "sugar", "sugars", "added-sugars", "lactose", "sucrose",
    "glucose", "fructose", "maltose"
]

carbohydrates_terms = [
    "carbohydrate", "starch", "maltodextrin", "glucides"
]

fiber_terms = [
    "fiber", "fibre", "soluble-fiber", "insoluble-fiber"
]

sodium_salt_terms = [
    "salt", "sodium"
]

vitamins_terms = [
    "vitamin", "folate", "niacin", "thiamin", "riboflavin", "pantothenic",
    "biotin", "cobalamin", "pyridoxine", "tocopherol", "retinol",
    "ascorbic", "cholecalciferol",
    # Added missing vitamin compounds
    "phylloquinone", "choline"
]

minerals_terms = [
    "calcium", "iron", "magnesium", "phosphorus", "potassium", "zinc",
    "copper", "manganese", "selenium", "iodine", "chromium", "molybdenum",
    "chloride", "fluoride", "sulphate", "sulfate", "bicarbonate", "nitrate",
    "fr-sulfate", "fr-sulfates", "silica"
]

amino_acids_terms = [
    "amino", "leucine", "isoleucine", "valine", "lysine", "methionine",
    "threonine", "tryptophan", "phenylalanine", "histidine", "arginine",
    "glycine", "serine", "proline", "alanine", "cysteine", "tyrosine",
    "aspartic", "glutamic", "asparagine", "glutamine"
]

# New categories
antioxidants_terms = [
    "beta-carotene", "lycopene", "lutein", "zeaxanthin", "anthocyanin",
    "flavonoid", "polyphenol", "resveratrol", "quercetin"
]

bioactive_compounds_terms = [
    "caffeine", "taurine", "inositol", "creatine", "carnitine",
    "coenzyme", "ubiquinone"
]

food_composition_terms = [
    "fruits-vegetables-nuts", "cocoa", "alcohol", "polyols",
    "erythritol", "ph", "carbon-footprint", "nutrition-score"
]

def categorise_nutrients(row: pd.Series) -> str:
    """
    Categorize nutrients based on their names with improved priority ordering.
    More specific categories are checked first to avoid misclassification.
    """
    name_lower = row["name"].lower()

    # Check most specific categories first
    if any(term in name_lower for term in amino_acids_terms):
        return "amino_acids"
    elif any(term in name_lower for term in antioxidants_terms):
        return "antioxidants"
    elif any(term in name_lower for term in bioactive_compounds_terms):
        return "bioactive_compounds"
    elif any(term in name_lower for term in vitamins_terms):
        return "vitamins"
    elif any(term in name_lower for term in minerals_terms):
        return "minerals"
    elif any(term in name_lower for term in energy_terms):
        return "energy"
    elif any(term in name_lower for term in protein_terms):
        return "protein"
    elif any(term in name_lower for term in sugars_terms):
        return "sugars"
    elif any(term in name_lower for term in carbohydrates_terms):
        return "carbohydrates"
    elif any(term in name_lower for term in fiber_terms):
        return "fiber"
    elif any(term in name_lower for term in sodium_salt_terms):
        return "sodium_salt"
    elif any(term in name_lower for term in fat_terms):
        return "fat"
    elif any(term in name_lower for term in food_composition_terms):
        return "food_composition"
    else:
        return "other"

nutrients_df["category"] = nutrients_df.apply(categorise_nutrients, axis=1)


Analyse nutrients by categories.


In [13]:
for category, group in nutrients_df.groupby("category"):
    print(f"\nCategory: \033[94m{category}\033[0m\n")
    for i, row in enumerate(group.itertuples(index=False), start=1):
        print_col(i, row.name, desc=f"{row.count:10,d} ({row.percent:5.2f}%)", sample=row.unit)



Category: antioxidants

  1. beta-carotene                            |        112 ( 0.00%) | g

Category: bioactive_compounds

  1. caffeine                                 |      4,353 ( 0.15%) | g
  2. taurine                                  |        341 ( 0.01%) | g
  3. inositol                                 |        127 ( 0.00%) | g

Category: carbohydrates

  1. carbohydrates                            |  2,490,622 (83.27%) | g
  2. starch                                   |        731 ( 0.02%) | g
  3. glucides assimilables                    |        669 ( 0.02%) | g
  4. maltodextrins                            |        236 ( 0.01%) | g

Category: energy

  1. energy                                   |  2,820,319 (94.29%) | kJ
  2. energy-kcal                              |  2,784,551 (93.10%) | kJ
  3. energy-kj                                |    368,607 (12.32%) | kJ
  4. energy                                   |      9,467 ( 0.32%) | kj
  5. energy-kj                

## 5. Nutrient Data Cleanup


The energy category in `nutrient_extract_df` is inconsistent, with duplicate labels (e.g., `energy`, `energy-kcal`, `energy-kj`) and mismatched units (e.g., `kJ`, `kj`, `g`). These inconsistencies lead to duplication and errors in analysis. A cleanup is required to standardize the labels, convert all values to a single unit (e.g., `kJ`), and consolidate the fields. The cleaned dataset will be saved in `data/nutrient_extract_cleaned`.


In [14]:
def clean_up_nutrient_extract(df: pd.DataFrame) -> pd.DataFrame:
    # Make a copy
    cleaned_df = df.copy()

    # Normalize units casing
    cleaned_df["unit"] = cleaned_df["unit"].str.strip().str.lower().replace({"kj": "kJ"})

    # Map alternative names to standard energy
    name_map = {
        "energy": "energy",
        "energy-kcal": "energy",
        "energy-kj": "energy",
        "energy-from-fat": "energy",
        "energy-g": "energy",   # likely invalid, but we still merge first
    }
    cleaned_df["name"] = cleaned_df["name"].replace(name_map)

    # Convert kcal to kJ (1 kcal = 4.184 kJ)
    mask_kcal = (df["name"].isin(["energy-kcal"])) | ((df["name"] == "energy") & (df["unit"] == "kcal"))
    cleaned_df.loc[mask_kcal, "100g"] = cleaned_df.loc[mask_kcal, "100g"] * 4.184
    cleaned_df.loc[mask_kcal, "unit"] = "kJ"

    # Drop rows where unit is nonsense (e.g., 'g' for energy)
    mask_bad_energy = (cleaned_df["name"] == "energy") & (cleaned_df["unit"] == "g")
    cleaned_df = cleaned_df[~mask_bad_energy]

    return cleaned_df

parts = nutrient_extract_df.map_partitions(clean_up_nutrient_extract)

with ProgressBar():
    parts.to_parquet("data/nutrient_extract_cleaned", engine="pyarrow", write_index=False, overwrite=True)


[########################################] | 100% Completed | 3.17 sms


Load the `data/nutrient_extract_cleaned` dataset.


In [15]:
nutrient_extract_cleaned_df = dd.read_parquet("data/nutrient_extract_cleaned", engine="pyarrow", dtype_backend="pyarrow")


Now, we construct the nutrient dataset again.


In [16]:
focus_nutrients = (
    nutrient_extract_cleaned_df.groupby(["name", "unit"])
    .size()
    .compute()
    .loc[lambda x: x >= 100]
    .sort_values(ascending=False)
)

nutrients_df = focus_nutrients.reset_index()
nutrients_df.columns = ["name", "unit", "count"]

nutrients_df["percent"] = nutrients_df["count"] / n_products * 100

nutrients_df["category"] = nutrients_df.apply(categorise_nutrients, axis=1)


Analyse nutrients by categories again to ensure that the data issue in energy category is fixed.


In [17]:
for category, group in nutrients_df.groupby("category"):
    print(f"\nCategory: \033[94m{category}\033[0m\n")
    for i, row in enumerate(group.itertuples(index=False), start=1):
        print_col(i, row.name, desc=f"{row.count:10,d} ({row.percent:5.2f}%)", sample=row.unit)



Category: antioxidants

  1. beta-carotene                            |        112 ( 0.00%) | g

Category: bioactive_compounds

  1. caffeine                                 |      4,353 ( 0.15%) | g
  2. taurine                                  |        341 ( 0.01%) | g
  3. inositol                                 |        127 ( 0.00%) | g

Category: carbohydrates

  1. carbohydrates                            |  2,490,622 (83.27%) | g
  2. starch                                   |        731 ( 0.02%) | g
  3. glucides assimilables                    |        669 ( 0.02%) | g
  4. maltodextrins                            |        236 ( 0.01%) | g

Category: energy

  1. energy                                   |  5,993,469 (200.38%) | kJ

Category: fat

  1. fat                                      |  2,301,562 (76.95%) | g
  2. saturated-fat                            |  2,010,883 (67.23%) | g
  3. cholesterol                              |    184,758 ( 6.18%) | g
  4. monounsatur

Saving `nutrients_df` to `data/nutrients`.


In [18]:
nutrients_df.to_csv("data/nutrients.csv", index=False)


## 6. Nutrient Outlier Detection


Compute summary statistics and perform outlier detection (IQR and MAD).


In [19]:
def summarise_nutrients(df: dd.DataFrame, nutrients: list[tuple[str, str]]):
    results = []

    for nutrient in nutrients:
        values = df.loc[(df["name"] == nutrient[0]) & (df["unit"] == nutrient[1]), "100g"]

        count = values.count().compute()
        median = values.quantile(0.5).compute()
        mean = values.mean().compute()
        std = values.std().compute()

        Q1 = values.quantile(0.25).compute()
        Q3 = values.quantile(0.75).compute()
        IQR = Q3 - Q1
        lower_iqr = max(0, Q1 - 1.5 * IQR)
        upper_iqr = Q3 + 1.5 * IQR
        outliers_iqr = ((values < lower_iqr) | (values > upper_iqr)).sum().compute()

        mad = (np.abs(values - median)).quantile(0.5).compute()
        scaled_mad = 1.4826 * mad
        lower_mad = max(0, median - 3 * scaled_mad)
        upper_mad = median + 3 * scaled_mad
        outliers_mad = ((values < lower_mad) | (values > upper_mad)).sum().compute()

        results.append({
            "Nutrient": nutrient,
            "Count": count,
            "Median": median,
            "Mean": mean,
            "Std": std,
            "Lower_MAD": lower_mad,
            "Lower_IQR": lower_iqr,
            "Upper_MAD": upper_mad,
            "Upper_IQR": upper_iqr,
            "Outliers_MAD": outliers_mad,
            "Outliers_IQR": outliers_iqr,
        })

    return pd.DataFrame(results)

nutrients_to_check = [
    ("energy", "kJ"),
    ("proteins", "g"),
    ("fat", "g"),
    ("carbohydrates", "g"),
    ("sugars", "g"),
    ("fiber", "g"),
    ("salt", "g"),
    ("sodium", "g"),
]

summarise_nutrients(nutrient_extract_cleaned_df, nutrients_to_check)


,Nutrient,Count,Median,Mean,Std,Lower_MAD,Lower_IQR,Upper_MAD,Upper_IQR,Outliers_MAD,Outliers_IQR
0,"(energy, kJ)",5993469,1205.000,-5.876774e+12,1.443587e+16,0,0,4260.638600,3765.600000,30986,46181
1,"(proteins, g)",2382737,8.890,2.335876e+04,3.593043e+07,0,0,35.087544,39.000000,70980,60421
2,"(fat, g)",2301562,13.330,3.846613e+01,3.296093e+04,0,0,62.611624,65.934999,69434,63918
3,"(carbohydrates, g)",2490622,29.850,5.535064e+01,3.216993e+04,0,0,141.712169,148.385000,10169,9681
4,"(sugars, g)",2183241,10.830,5.380613e+26,7.950283e+29,0,0,53.217534,78.075005,156415,36658
5,"(fiber, g)",995152,3.300,1.004872e+14,1.002433e+17,0,0,12.640380,14.349999,63311,48545
6,"(salt, g)",2025188,0.885,2.643579e+00,2.783007e+02,0,0,4.332045,3.761667,104998,131645
7,"(sodium, g)",2162812,0.354,1.029252e+00,1.077210e+02,0,0,1.715027,1.504667,114678,140893


There are many outliers. Let’s peek into the proteins data by viewing 3 rows from each partition.


In [20]:
nutrient_extract_cleaned_df.map_partitions(
    lambda df:
        df[(df["100g"] > 100) & (df["name"] == "proteins")].head(3)
).compute()


,code,name,100g,unit
4517,0008725247045,proteins,119.0,g
12133,0010082100154,proteins,139.0,g
13488,0010300843290,proteins,131.0,g
154753,3228470064774,proteins,206.0,g
1437782,4015051011567,proteins,159.0,g
2281570,8411555110090,proteins,203.0,g
174828,0055742357509,proteins,130.0,g
257206,7501791634298,proteins,291.0,g
270005,7613331747129,proteins,190.0,g
31551,0041483040138,proteins,1700.0,g


Number of rows with invalid proteins data.


In [21]:
n_invalid_proteins = nutrient_extract_cleaned_df[
    (nutrient_extract_cleaned_df["100g"] >= 100) & (nutrient_extract_cleaned_df["name"] == "proteins")
].shape[0].compute()

print(f"Found {n_invalid_proteins:,} rows with invalid proteins data.")


Found 2,108 rows with invalid proteins data.


Totally hilarious protein numbers.


In [22]:
nutrient_extract_cleaned_df[
    (nutrient_extract_cleaned_df["100g"] > 10000) & (nutrient_extract_cleaned_df["name"] == "proteins")
].compute()


,code,name,100g,unit
1308917,08041902,proteins,100000.0,g
2853109,2293160029525,proteins,40939.0,g
3743045,0750894613007,proteins,100000000.0,g
3794358,8542024546710,proteins,50000000.0,g
2750364,4606068638368,proteins,55462510592.0,g
3169348,8698720571034,proteins,8578293.0,g
4491768,7009189960009,proteins,10000000.0,g


## 7. Creation of Nutritional Profiles


Create product nutrients dataset and save into `data\product_nutrients`.


In [23]:
def pivot_nutrients(nutrients_df: dd.DataFrame, focus_nutrients: pd.Series):
    names_only = focus_nutrients.index.get_level_values("name")

    # Convert names_only to list of unique strings
    categories = list(pd.Series(names_only).unique())

    # Filter and set 'name' as categorical with known categories
    filtered = nutrients_df[nutrients_df["name"].isin(categories)].assign(
        name=lambda df: df["name"].astype("category").cat.set_categories(categories)
    )

    # Lazy pivot
    pivoted = filtered.pivot_table(
        index="code",
        columns="name",
        values="100g",
        aggfunc="first"
    )

    return pivoted

pivot_df = pivot_nutrients(nutrient_extract_cleaned_df, focus_nutrients)

with ProgressBar():
    pivot_df.columns = pivot_df.columns.astype(str)
    pivot_df.to_parquet("data/product_nutrients", engine="pyarrow", write_index=True, overwrite=True)


[########################################] | 100% Completed | 12.11 ss


Load the `data/product_nutrients` dataset.


In [24]:
product_nutrients_df = dd.read_parquet("data/product_nutrients", engine="pyarrow", dtype_backend="pyarrow")


Add `nutriscore_grade` and `nutriscore_score` into the `product_nutrients_df`.


In [25]:
product_nutrients_df = product_nutrients_df.merge(
    df[["code", "nutriscore_grade", "nutriscore_score"]],
    on="code",
    how="left",
)

with ProgressBar():
    product_nutrients_df.to_parquet("data/product_nutrients", engine="pyarrow", write_index=True)

[########################################] | 100% Completed | 9.92 ss


Load the `data/product_nutrients` dataset again.


In [26]:
product_nutrients_df = dd.read_parquet("data/product_nutrients", engine="pyarrow", dtype_backend="pyarrow")


Check unique values for `nutriscore_grade`.


In [27]:
product_nutrients_df["nutriscore_grade"].unique().compute()


0                 c
1                 a
0                 e
1                 d
2                 b
3              <NA>
0    not-applicable
1           unknown
Name: nutriscore_grade, dtype: string

View top 3 rows in `product_nutrients_df`.


In [28]:
product_nutrients_df.head(3)


,code,energy,carbohydrates,proteins,fat,sugars,sodium,salt,saturated-fat,nova-group,...,unsaturated-fat,inositol,erythritol,nutrition-score-fr,maltose,fr-sulfates,beta-carotene,behenic-acid,nutriscore_grade,nutriscore_score
0,00000016,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,not-applicable,<NA>
1,00000020,1430.0,31.4,57.099998,5.71,8.57,0.714,1.79,<NA>,4.0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,unknown,<NA>
2,00000022,1630.0,35.200001,37.0,16.700001,1.85,0.352,0.88,6.48,4.0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,c,9


Create nutritional profiles dataset and save into `data\product_nutrient_profiles`.


In [29]:
def create_nutritional_profiles(df: pd.DataFrame) -> pd.DataFrame:
    profiles_df = pd.DataFrame(index=df.index)

    profiles_df["code"] = df["code"]

    # Macronutrient ratios
    total_macros = df['proteins'] + df['fat'] + df['carbohydrates']

    profiles_df['protein_ratio'] = df['proteins'] / total_macros
    profiles_df['fat_ratio'] = df['fat'] / total_macros
    profiles_df['carb_ratio'] = df['carbohydrates'] / total_macros

    # Macronutrient profile categories
    conditions = [
        (profiles_df['protein_ratio'] > 0.4).fillna(False).astype(bool),  # High protein
        (profiles_df['fat_ratio'] > 0.4).fillna(False).astype(bool),      # High fat
        (profiles_df['carb_ratio'] > 0.6).fillna(False).astype(bool),     # High carb
    ]
    choices = ['high_protein', 'high_fat', 'high_carb'] # else Balanced
    profiles_df['macro_profile'] = np.select(conditions, choices, default='balanced')

    # Energy density
    energy_kcal = df['energy'] / 4.184
    profiles_df['energy_density'] = energy_kcal

    # Energy density categories
    profiles_df['energy_category'] = pd.cut(
        energy_kcal,
        bins=[0, 150, 300, 500, float('inf')],
        labels=['low_energy', 'moderate_energy', 'high_energy', 'very_high_energy']
    )

    # Sugar
    profiles_df['sugar_to_carb_ratio'] = df['sugars'] / df['carbohydrates']
    profiles_df['high_sugar'] = profiles_df['sugar_to_carb_ratio'] > 0.5

    # Fiber - fiber > 6g per 100g
    profiles_df['fiber_g'] = df['fiber']
    profiles_df['high_fiber'] = df['fiber'] > 6

    # Salt - sodium > 0.6g or salt > 1.5g per 100g
    profiles_df['sodium_g'] = df['sodium']
    profiles_df['salt_g'] = df['salt']
    profiles_df['high_salt'] = (df['sodium'] > 0.6) | (df['salt'] > 1.5)

    # Micronutrient richness score
    vitamin_cols = nutrients_df[nutrients_df["category"] == "vitamins"]["name"].unique()
    mineral_cols = nutrients_df[nutrients_df["category"] == "minerals"]["name"].unique()
    micronutrient_cols = [*vitamin_cols, *mineral_cols]
    profiles_df['vitamin_richness'] = (df[vitamin_cols].notna().sum(axis=1) / len(vitamin_cols)) * 100
    profiles_df['mineral_richness'] = (df[mineral_cols].notna().sum(axis=1) / len(vitamin_cols)) * 100
    profiles_df['micronutrient_richness'] = (df[micronutrient_cols].notna().sum(axis=1) / len(micronutrient_cols)) * 100

    # Nova Group
    profiles_df['processing_level'] = df['nova-group']
    profiles_df['ultra_processed'] = df['nova-group'] == 4

    # Nutrient Density Score (micronutrients per calorie)
    energy_kcal = df['energy'] / 4.184  # Convert kJ to kcal
    micronutrient_count = df[micronutrient_cols].notna().sum(axis=1)
    profiles_df['nutrient_density_score'] = micronutrient_count / (energy_kcal + 1)  # +1 to avoid division by zero

    # Glycemic Impact Indicator (sugar-to-fiber ratio)
    fiber_safe = df['fiber'].fillna(0.1)  # Avoid division by zero
    profiles_df['glycemic_impact'] = df['sugars'] / fiber_safe
    profiles_df['high_glycemic_impact'] = profiles_df['glycemic_impact'] > 10  # High sugar, low fiber

    # Satiety Score (protein + fiber combination)
    profiles_df['satiety_score'] = (df['proteins'] * 2 + df['fiber']) / 3
    profiles_df['high_satiety'] = profiles_df['satiety_score'] > 8  # High satiety potential

    # Nutri-Score
    profiles_df['nutri_score'] = df['nutriscore_score']
    profiles_df['nutri_grade'] = df['nutriscore_grade']

    return profiles_df

parts = product_nutrients_df.map_partitions(create_nutritional_profiles)

with ProgressBar():
    parts.to_parquet("data/product_nutrient_profiles", engine="pyarrow", write_index=True, overwrite=True)


[########################################] | 100% Completed | 1.30 sms


Load the `data/product_nutrient_profiles` dataset.


In [30]:
nutrient_profiles_df = dd.read_parquet("data/product_nutrient_profiles", engine="pyarrow", dtype_backend="pyarrow")


View top 3 rows in `nutrient_profiles_df`.


In [31]:
nutrient_profiles_df.head(3)


,code,protein_ratio,fat_ratio,carb_ratio,macro_profile,energy_density,energy_category,sugar_to_carb_ratio,high_sugar,fiber_g,...,micronutrient_richness,processing_level,ultra_processed,nutrient_density_score,glycemic_impact,high_glycemic_impact,satiety_score,high_satiety,nutri_score,nutri_grade
0,00000016,<NA>,<NA>,<NA>,balanced,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.0,1.0,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,not-applicable
1,00000020,0.606093,0.060609,0.333298,high_protein,341.778203,high_energy,0.27293,False,14.3,...,13.888889,4.0,True,0.014587,0.599301,False,42.833332,True,<NA>,unknown
2,00000022,0.416198,0.187852,0.395951,high_protein,389.57935,high_energy,0.052557,False,18.5,...,0.0,4.0,True,0.0,0.1,False,30.833333,True,9,c


Analyse product's nutritional profiles.


In [32]:
def analyze_column(df: pd.DataFrame, column: str) -> None:
    """
    Analyse a column in a dataframe.
    - For numeric columns: prints mean and median.
    - For categorical columns: prints distribution counts and percentages.
    """
    if pd.api.types.is_numeric_dtype(df[column]):
        mean_val = df[column].mean().compute()
        median_val = df[column].median_approximate().compute()
        print(f"{column.replace('_', ' ').title()} - Mean: {mean_val:.3f}, Median: {median_val:.3f}")
    else:
        counts = df[column].value_counts().compute()
        total = counts.sum()
        print(f"{column.replace('_', ' ').title()} Distribution:")
        for cat, count in counts.items():
            pct = (count / total) * 100
            print(f"  {cat}: {count:,} products ({pct:.1f}%)")

analyze_column(nutrient_profiles_df, "protein_ratio")
analyze_column(nutrient_profiles_df, "fat_ratio")
analyze_column(nutrient_profiles_df, "carb_ratio")
analyze_column(nutrient_profiles_df, "macro_profile")

print()

analyze_column(nutrient_profiles_df, "energy_density")
analyze_column(nutrient_profiles_df, "energy_category")

print()

analyze_column(nutrient_profiles_df, "sugar_to_carb_ratio")
analyze_column(nutrient_profiles_df, "high_sugar")

print()

analyze_column(nutrient_profiles_df, "fiber_g")
analyze_column(nutrient_profiles_df, "high_fiber")

print()

analyze_column(nutrient_profiles_df, "sodium_g")
analyze_column(nutrient_profiles_df, "salt_g")
analyze_column(nutrient_profiles_df, "high_salt")

print()

analyze_column(nutrient_profiles_df, "vitamin_richness")
analyze_column(nutrient_profiles_df, "mineral_richness")
analyze_column(nutrient_profiles_df, "micronutrient_richness")

print()

analyze_column(nutrient_profiles_df, "processing_level")
analyze_column(nutrient_profiles_df, "ultra_processed")

print()

analyze_column(nutrient_profiles_df, "nutrient_density_score")

print()

analyze_column(nutrient_profiles_df, "glycemic_impact")
analyze_column(nutrient_profiles_df, "high_glycemic_impact")

print()

analyze_column(nutrient_profiles_df, "satiety_score")
analyze_column(nutrient_profiles_df, "high_satiety")

print()

analyze_column(nutrient_profiles_df, "nutri_score")
analyze_column(nutrient_profiles_df, "nutri_grade")

Protein Ratio - Mean: 0.231, Median: 0.150
Fat Ratio - Mean: 0.259, Median: 0.221
Carb Ratio - Mean: 0.510, Median: 0.584
Macro Profile Distribution:
  high_fat: 355,501 products (11.9%)
  high_protein: 378,974 products (12.7%)
  balanced: 1,262,097 products (42.2%)
  high_carb: 994,520 products (33.2%)

Energy Density - Mean: 5017009769.348, Median: 260.038
Energy Category Distribution:
  high_energy: 902,868 products (31.9%)
  low_energy: 912,248 products (32.2%)
  moderate_energy: 673,479 products (23.8%)
  very_high_energy: 0 products (0.0%)
  high_energy: 0 products (0.0%)
  low_energy: 0 products (0.0%)
  moderate_energy: 0 products (0.0%)
  very_high_energy: 341,777 products (12.1%)

Sugar To Carb Ratio - Mean: 576935176222.382, Median: 0.563
High Sugar Distribution:
  False: 1,018,638 products (46.8%)
  True: 1,160,212 products (53.2%)

Fiber G - Mean: 100484645560875.844, Median: 2.800
High Fiber Distribution:
  False: 764,645 products (76.8%)
  True: 230,532 products (23.2%)
